<a href="https://colab.research.google.com/github/BTUCYB/NLP-LLM-Agent/blob/main/NLP_CYB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Cert_Dataset/
import os
print("当前目录:", os.getcwd)

/content/drive/MyDrive/Cert_Dataset
当前目录: <built-in function getcwd>


In [ ]:
!ls -l r4.2_clean_{0..8}.zip

-rw------- 1 root root 838860800 May  4 13:30 r4.2_clean_0.zip
-rw------- 1 root root 838860800 May  4 13:30 r4.2_clean_1.zip
-rw------- 1 root root 838860800 May  4 13:30 r4.2_clean_2.zip
-rw------- 1 root root 838860800 May  4 13:30 r4.2_clean_3.zip
-rw------- 1 root root 838860800 May  4 13:30 r4.2_clean_4.zip
-rw------- 1 root root 838860800 May  4 13:30 r4.2_clean_5.zip
-rw------- 1 root root 838860800 May  4 13:30 r4.2_clean_6.zip
-rw------- 1 root root 838860800 May  4 13:30 r4.2_clean_7.zip
-rw------- 1 root root 469887245 May  4 13:30 r4.2_clean_8.zip


In [ ]:
!cat r4.2_clean_0.zip \
r4.2_clean_1.zip \
r4.2_clean_2.zip \
r4.2_clean_3.zip \
r4.2_clean_4.zip \
r4.2_clean_5.zip \
r4.2_clean_6.zip \
r4.2_clean_7.zip \
r4.2_clean_8.zip > full_certdataset.zip

In [ ]:
!ls -lh full_certdataset.zip

-rw------- 1 root root 6.7G May  7 13:55 full_certdataset.zip


In [ ]:
!mkdir -p /content/drive/MyDrive/Cert_Dataset/full_certdataset

In [ ]:
!7z x full_certdataset.zip -o/content/drive/MyDrive/Cert_Dataset/full_certdataset


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 7180773645 bytes (6849 MiB)

Extracting archive: full_certdataset.zip
--
Path = full_certdataset.zip
Type = zip
Physical Size = 7180773645
64-bit = +

  0%      0% 2 - r4.2_clean/device_cleaned.csv                                        0% 3        0% 3 - r4.2_clean/email_2010_part1.csv                                          1% 3 - r4.2_clean/email_2010_part1.csv                                          2% 3 - r4.2_clean/email_2010_part1.csv

In [ ]:
!ls -lh /content/drive/MyDrive/Cert_Dataset/full_certdataset

total 4.0K
drwx------ 2 root root 4.0K Apr 15 09:23 r4.2_clean


## Data Preprocessing
### Target:complete data screening & format standardization

In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/Cert_Dataset/full_certdataset/r4.2_clean/device_cleaned.csv'
df_head = pd.read_csv(file_path, nrows=5, header=None)
print('文件的前几行内容：')
print(df_head.to_csv(sep='\t', na_rep='nan'))

文件的前几行内容：
	0	1	2	3	4	5	6	7
0	id	date	user	pc	activity	hour	dayofweek	activity_binary
1	{J1S3-L9UU75BQ-7790ATPL}	2010-01-02 07:21:06	MOH0273	PC-6699	Connect	7	5	1
2	{N7B5-Y7BB27SI-2946PUJK}	2010-01-02 07:37:41	MOH0273	PC-6699	Disconnect	7	5	0
3	{U1V9-Z7XT67KV-5649MYHI}	2010-01-02 07:59:11	HPH0075	PC-2417	Connect	7	5	1
4	{H0Z7-E6GB57XZ-1603MOXD}	2010-01-02 07:59:49	IIW0249	PC-0843	Connect	7	5	1



In [ ]:
import pandas as pd
import os
from google.colab import drive

# 再次挂载 Google Drive
drive.mount('/content/drive')

# 根据实际列名调整Required_Cols
Required_Cols = ['user', 'date', 'activity', 'activity_binary']
output_path = '/content/drive/MyDrive/processed/filtered_data.csv'
chunksize = 60000
directory_path = '/content/drive/MyDrive/Cert_Dataset/full_certdataset/r4.2_clean'

# 确保输出文件夹存在
output_dir = os.path.dirname(output_path)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"创建文件夹: {output_dir}")

csv_files_found = False
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        csv_files_found = True
        file_path = os.path.join(directory_path, filename)
        print(f"正在处理文件: {file_path}")
        for chunk in pd.read_csv(
            file_path,
            header=0,
            chunksize=chunksize,
            encoding='latin1'
        ):
            available_cols = chunk.columns
            valid_Required_Cols = [col for col in Required_Cols if col in available_cols]
            # 选择需要的列
            chunk = chunk[valid_Required_Cols]
            # 根据实际列名调整dropna和fillna操作
            if 'activity' in valid_Required_Cols:
                chunk = chunk.dropna(subset=['activity'])
            if 'activity_binary' in valid_Required_Cols:
                chunk['activity_binary'] = chunk['activity_binary'].fillna(0).astype('int8')

            header = not os.path.exists(output_path)
            try:
                chunk.to_csv(output_path, mode='a', header=header, index=False)
                print(f"成功写入 {len(chunk)} 行到 {output_path}")
            except Exception as e:
                print(f"写入文件时出错: {e}")

if not csv_files_found:
    print(f"未在 {directory_path} 目录下找到 CSV 文件，请检查文件扩展名或路径是否正确。")

print("pre - processing finish successfully! output file:", output_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
正在处理文件: /content/drive/MyDrive/Cert_Dataset/full_certdataset/r4.2_clean/device_cleaned.csv
成功写入 60000 行到 /content/drive/MyDrive/processed/filtered_data.csv
成功写入 60000 行到 /content/drive/MyDrive/processed/filtered_data.csv
成功写入 60000 行到 /content/drive/MyDrive/processed/filtered_data.csv
成功写入 60000 行到 /content/drive/MyDrive/processed/filtered_data.csv
成功写入 60000 行到 /content/drive/MyDrive/processed/filtered_data.csv
成功写入 60000 行到 /content/drive/MyDrive/processed/filtered_data.csv
成功写入 45380 行到 /content/drive/MyDrive/processed/filtered_data.csv
正在处理文件: /content/drive/MyDrive/Cert_Dataset/full_certdataset/r4.2_clean/email_2010_part1.csv
成功写入 60000 行到 /content/drive/MyDrive/processed/filtered_data.csv
成功写入 60000 行到 /content/drive/MyDrive/processed/filtered_data.csv
成功写入 60000 行到 /content/drive/MyDrive/processed/filtered_data.csv
成功写入 60000 行到 /content/drive/MyDrive/

In [ ]:
!head -n 1 /content/drive/MyDrive/processed/filtered_data.csv

user,date,activity,activity_binary


In [ ]:
!wc -l /content/drive/MyDrive/processed/filtered_data.csv

33175603 /content/drive/MyDrive/processed/filtered_data.csv


### Deep Cleaning

In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/processed/filtered_data.csv'
# 读取文件的前几行
df_head = pd.read_csv(file_path, nrows=5)
print('文件的列名：')
print(df_head.columns)

文件的列名：
Index(['user', 'date', 'activity', 'activity_binary'], dtype='object')


In [ ]:
import re
from tqdm import tqdm
import pandas as pd
import os

def clean_text(text):
    # 将 text 转换为字符串
    text = str(text)
    # 删除非ASCII字符
    text = text.encode('ascii', 'ignore').decode()
    # 标准化日期格式（示例）
    text = re.sub(r'\d{4}-\d{2}-\d{2}', '[DATE]', text)
    # 删除特殊符号
    return re.sub(r'[^a-zA-Z0-9\s.,?!]', '', text)

# 分块处理清洗
chunksize = 50000
input_file_path = '/content/drive/MyDrive/processed/filtered_data.csv'
cleaned_path = '/content/drive/MyDrive/processed/cleaned_data.csv'

# 检查输入文件是否存在
if not os.path.exists(input_file_path):
    print(f"输入文件 {input_file_path} 不存在，请检查路径。")
    exit(0)

# 检查输出文件是否存在
file_exists = os.path.exists(cleaned_path)

for chunk in tqdm(pd.read_csv(input_file_path, chunksize=chunksize)):
    # 列名 'activity' 进行清洗
    chunk['activity'] = chunk['activity'].apply(clean_text)
    header = not file_exists  # 根据文件是否存在决定是否写入表头
    try:
        chunk.to_csv(cleaned_path, mode='a', header=header, index=False)
    except:
        print(f"写入文件 {cleaned_path} 时出错，请检查文件权限或格式。")

print("清洗完成！输出文件：", cleaned_path)

664it [03:06,  3.56it/s]

清洗完成！输出文件： /content/drive/MyDrive/processed/cleaned_data.csv


### TF-IDF Feature Extraction

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# 读取清洗后的数据文件，设置 low_memory=False 来处理混合数据类型警告
cleaned_data_path = '/content/drive/MyDrive/processed/cleaned_data.csv'
cleaned_df = pd.read_csv(cleaned_data_path, low_memory=False)

texts = cleaned_df['activity'].tolist()

# 处理 np.nan 值，将其替换为空字符串
texts = ['' if pd.isna(text) else text for text in texts]

# 进行 TF-IDF 特征提取
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

# 获取特征名称
feature_names = vectorizer.get_feature_names_out()
print("特征名称:", feature_names)

# 打印 TF-IDF 矩阵的形状
print("TF-IDF 矩阵形状:", tfidf_matrix.shape)

特征名称: ['activity' 'connect' 'disconnect' 'logoff' 'logon']
TF-IDF 矩阵形状: (67131216, 5)


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import save_npz
import os

# 读取清洗后的数据文件，设置 low_memory=False 来处理混合数据类型警告
cleaned_data_path = '/content/drive/MyDrive/processed/cleaned_data.csv'
try:
    cleaned_df = pd.read_csv(cleaned_data_path, low_memory=False)
except FileNotFoundError:
    print(f"错误：未找到清洗后的数据文件 {cleaned_data_path}，请检查文件路径。")
else:
    texts = cleaned_df['activity'].tolist()

    # 处理 np.nan 值，将其替换为空字符串
    texts = ['' if pd.isna(text) else text for text in texts]

    # 进行 TF-IDF 特征提取
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts)

    # 获取特征名称
    feature_names = vectorizer.get_feature_names_out()
    print("特征名称:", feature_names)

    # 打印 TF-IDF 矩阵的形状
    print("TF-IDF 矩阵形状:", tfidf_matrix.shape)

    # 定义保存路径
    save_dir = '/content/drive/MyDrive/features/tfidf'
    # 检查保存目录是否存在，不存在则创建
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    save_path = os.path.join(save_dir, 'chunk_0.npz')

    try:
        # 保存 TF-IDF 矩阵为 .npz 文件
        save_npz(save_path, tfidf_matrix)
        print(f"TF-IDF 矩阵已成功保存到 {save_path}")
    except Exception as e:
        print(f"错误：保存 TF-IDF 矩阵时出现问题：{e}，请检查保存路径的权限或者磁盘空间。")

特征名称: ['activity' 'connect' 'disconnect' 'logoff' 'logon']
TF-IDF 矩阵形状: (67131216, 5)
TF-IDF 矩阵已成功保存到 /content/drive/MyDrive/features/tfidf/chunk_0.npz


### BERT Feature Extraction


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
os.environ["TF_FORCE_UNIFIED_MEMORY"] = "false"
os.environ["TF_ENABLE_GPU_GARBAGE_COLLECTION"] = "false"
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
os.environ["PYTHONOPTS"] = "-Xfrozen_modules=off"

import tensorflow_hub as hub
import tensorflow_text as text  # 必须安装
import pandas as pd
import numpy as np
from google.colab import drive
import gc

# 挂载 Google Drive
try:
    drive.mount('/content/drive')
except Exception as e:
    print(f"挂载Drive时出现问题: {e}")

# 定义清洗后数据文件的路径
cleaned_path = '/content/drive/MyDrive/processed/cleaned_data.csv'

# 检查文件是否存在
if not os.path.exists(cleaned_path):
    print(f"文件 {cleaned_path} 不存在，请检查路径或重新生成文件。")
else:
    # 加载预训练模型
    bert_preprocess = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
    bert_encoder = hub.load("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

    # 限制读取行数进行测试
    nrows = 100  # 可以根据需要调整
    texts = pd.read_csv(cleaned_path, nrows=nrows, encoding='utf-8')

    # 检查是否存在 'activity' 列
    if 'activity' not in texts.columns:
        print(f"数据中不存在 'activity' 列，实际列名是: {texts.columns}")
    else:
        texts = texts['activity']

        # 检查并处理缺失值
        if texts.isnull().any():
            texts = texts.dropna()
            print("已移除缺失值的行。")

        texts = texts.tolist()

        # 额外的数据预处理
        texts = [str(text).lower().strip() for text in texts]

        # 分批处理（每批50条）
        batch_size = 50

        # 定义保存特征的目录
        save_dir = '/content/drive/MyDrive/features/bert'
        # 检查目录是否存在，不存在则创建
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)

        for i in range(0, len(texts), batch_size):
            try:
                batch = texts[i:i+batch_size]
                preprocessed = bert_preprocess(batch)
                outputs = bert_encoder(preprocessed)
                cls_vectors = outputs['pooled_output'].numpy()
                save_path = os.path.join(save_dir, f'chunk_{i}.npy')
                np.save(save_path, cls_vectors)

                # 手动释放不再使用的变量
                del batch, preprocessed, outputs, cls_vectors
                gc.collect()
                gc.collect()  # 多次调用垃圾回收
            except Exception as e:
                print(f"处理批次 {i} 到 {i+batch_size} 时出现错误: {e}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np

# 定义.npy文件的路径
file_path = '/content/drive/MyDrive/features/bert/chunk_0.npy'

# 加载.npy文件
data = np.load(file_path)

# 查看数据的形状
print(f"数据形状: {data.shape}")

# 查看数据的前几行（假设数据是二维的）
if data.ndim == 2:
    num_rows_to_view = min(5, data.shape[0])  # 查看前5行或数据的所有行（如果少于5行）
    print(f"数据的前 {num_rows_to_view} 行内容:")
    print(data[:num_rows_to_view])
else:
    print("数据不是二维的，可能需要根据实际情况调整查看方式。")

数据形状: (50, 768)
数据的前 5 行内容:
[[-0.74876195 -0.11396315  0.72913677 ...  0.6528602  -0.4941778
   0.7876105 ]
 [-0.9090212  -0.29025632 -0.6612839  ... -0.35183626 -0.6340631
   0.89593047]
 [-0.74876195 -0.11396315  0.72913677 ...  0.6528602  -0.4941778
   0.7876105 ]
 [-0.74876195 -0.11396315  0.72913677 ...  0.6528602  -0.4941778
   0.7876105 ]
 [-0.9090212  -0.29025632 -0.6612839  ... -0.35183626 -0.6340631
   0.89593047]]


In [ ]:
import numpy as np

# 定义.npy文件的路径
file_path = '/content/drive/MyDrive/features/bert/chunk_50.npy'

# 加载.npy文件
data = np.load(file_path)

# 查看数据的形状
print(f"数据形状: {data.shape}")

# 查看数据的前几行（假设数据是二维的）
if data.ndim == 2:
    num_rows_to_view = min(5, data.shape[0])  # 查看前5行或数据的所有行（如果少于5行）
    print(f"数据的前 {num_rows_to_view} 行内容:")
    print(data[:num_rows_to_view])
else:
    print("数据不是二维的，可能需要根据实际情况调整查看方式。")

数据形状: (50, 768)
数据的前 5 行内容:
[[-0.74876195 -0.11396315  0.72913677 ...  0.6528602  -0.4941778
   0.7876105 ]
 [-0.9090212  -0.29025632 -0.6612839  ... -0.35183626 -0.6340631
   0.89593047]
 [-0.9090212  -0.29025632 -0.6612839  ... -0.35183626 -0.6340631
   0.89593047]
 [-0.9090212  -0.29025632 -0.6612839  ... -0.35183626 -0.6340631
   0.89593047]
 [-0.74876195 -0.11396315  0.72913677 ...  0.6528602  -0.4941778
   0.7876105 ]]


### Feature Fusion



In [ ]:
import os
import numpy as np
from scipy.sparse import load_npz, save_npz
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer, AutoModel
import torch

# 禁用调试相关警告
os.environ["PYDEVD_DISABLE_FILE_VALIDATION"] = "1"
os.environ["PYTHONOPERATIONS"] = "-Xfrozen_modules=off"
# 禁用 GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


def extract_tfidf(texts, output_dir, index):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts)
    tfidf_file = os.path.join(output_dir, f'chunk_{index}.npz')
    save_npz(tfidf_file, tfidf_matrix)
    print(f"TF-IDF 特征已保存到 {tfidf_file}")


def extract_bert(texts, output_dir, index):
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    model = AutoModel.from_pretrained('bert-base-uncased')
    bert_features = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt')
        outputs = model(**inputs)
        bert_features.append(outputs.last_hidden_state.mean(dim=1).detach().numpy())
    bert_features = np.vstack(bert_features)
    bert_file = os.path.join(output_dir, f'chunk_{index}.npy')
    np.save(bert_file, bert_features)
    print(f"BERT 特征已保存到 {bert_file}")


tfidf_dir = '/content/drive/MyDrive/features/tfidf'
bert_dir = '/content/drive/MyDrive/features/bert'
output_dir = '/content/drive/MyDrive/features/hybrid'

# 检查输出目录是否存在，如果不存在则创建
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 检查 tfidf_dir 目录是否存在
if not os.path.exists(tfidf_dir):
    print(f"目录 {tfidf_dir} 不存在，请检查。")
    exit(1)

# 检查 bert_dir 目录是否存在
if not os.path.exists(bert_dir):
    print(f"目录 {bert_dir} 不存在，请检查。")
    exit(1)

tfidf_files = sorted(os.listdir(tfidf_dir))
bert_files = sorted(os.listdir(bert_dir))

if len(tfidf_files) != len(bert_files):
    print("tfidf 和 bert 文件数量不匹配，调整文件数量。")
    min_len = min(len(tfidf_files), len(bert_files))
    # 删除多余的 TF-IDF 文件
    if len(tfidf_files) > min_len:
        for file in tfidf_files[min_len:]:
            file_path = os.path.join(tfidf_dir, file)
            os.remove(file_path)
            print(f"已删除多余的 TF-IDF 文件: {file_path}")
    # 删除多余的 BERT 文件
    if len(bert_files) > min_len:
        for file in bert_files[min_len:]:
            file_path = os.path.join(bert_dir, file)
            os.remove(file_path)
            print(f"已删除多余的 BERT 文件: {file_path}")

    # 重新获取文件列表
    tfidf_files = sorted(os.listdir(tfidf_dir))
    bert_files = sorted(os.listdir(bert_dir))

    # 重新提取特征
    print("重新提取特征。")
    # 假设数据文件路径
    data_path = '/content/drive/MyDrive/processed/cleaned_data.csv'
    try:
        df = pd.read_csv(data_path, low_memory=False)  # 设置 low_memory=False 避免数据类型警告
        texts = df['activity'].tolist()
        chunk_size = 1000  # 假设每个 chunk 包含 1000 个样本
        num_chunks = len(texts) // chunk_size + (1 if len(texts) % chunk_size != 0 else 0)
        for i in range(num_chunks):
            start = i * chunk_size
            end = start + chunk_size
            chunk_texts = texts[start:end]
            extract_tfidf(chunk_texts, tfidf_dir, i)
            extract_bert(chunk_texts, bert_dir, i)
        tfidf_files = sorted(os.listdir(tfidf_dir))
        bert_files = sorted(os.listdir(bert_dir))
    except FileNotFoundError:
        print(f"数据文件 {data_path} 未找到，请检查路径。")
        exit(1)

for i in range(len(tfidf_files)):
    tfidf_file = os.path.join(tfidf_dir, tfidf_files[i])
    bert_file = os.path.join(bert_dir, bert_files[i])

    # 检查 tfidf 文件是否存在
    if not os.path.exists(tfidf_file):
        print(f"文件 {tfidf_file} 不存在，请检查。")
        continue

    # 检查 bert 文件是否存在
    if not os.path.exists(bert_file):
        print(f"文件 {bert_file} 不存在，请检查。")
        continue

    try:
        # 加载 TF-IDF
        tfidf = load_npz(tfidf_file).toarray()
        # 加载 BERT
        bert = np.load(bert_file)

        # 检查样本数量是否匹配
        if tfidf.shape[0] != bert.shape[0]:
            print(f"文件 {tfidf_file} 和 {bert_file} 的样本数量不匹配，重新提取该 chunk 的特征。")
            data_path = '/content/drive/MyDrive/processed/cleaned_data.csv'
            try:
                df = pd.read_csv(data_path, low_memory=False)
                texts = df['activity'].tolist()
                chunk_size = 1000
                start = i * chunk_size
                end = start + chunk_size
                chunk_texts = texts[start:end]
                extract_tfidf(chunk_texts, tfidf_dir, i)
                extract_bert(chunk_texts, bert_dir, i)
                # 重新加载特征
                tfidf = load_npz(tfidf_file).toarray()
                bert = np.load(bert_file)
            except FileNotFoundError:
                print(f"数据文件 {data_path} 未找到，请检查路径。")
                continue

        # 水平拼接
        hybrid = np.hstack([tfidf, bert])
        output_file = os.path.join(output_dir, f'hybrid_{i}.npy')
        np.save(output_file, hybrid)
        print(f"成功保存融合特征到 {output_file}")
    except ValueError as ve:
        print(f"值错误: 在处理 {tfidf_file} 和 {bert_file} 时出现维度不匹配或其他值错误: {ve}")
    except FileNotFoundError as fnfe:
        print(f"文件未找到错误: {fnfe}")
    except Exception as e:
        print(f"处理文件时出错: {e}")

tfidf 和 bert 文件数量不匹配，调整文件数量。
已删除多余的 BERT 文件: /content/drive/MyDrive/features/bert/chunk_50.npy
重新提取特征。
TF-IDF 特征已保存到 /content/drive/MyDrive/features/tfidf/chunk_0.npz


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BERT 特征已保存到 /content/drive/MyDrive/features/bert/chunk_0.npy
TF-IDF 特征已保存到 /content/drive/MyDrive/features/tfidf/chunk_1.npz
BERT 特征已保存到 /content/drive/MyDrive/features/bert/chunk_1.npy
TF-IDF 特征已保存到 /content/drive/MyDrive/features/tfidf/chunk_2.npz
BERT 特征已保存到 /content/drive/MyDrive/features/bert/chunk_2.npy
TF-IDF 特征已保存到 /content/drive/MyDrive/features/tfidf/chunk_3.npz
BERT 特征已保存到 /content/drive/MyDrive/features/bert/chunk_3.npy
TF-IDF 特征已保存到 /content/drive/MyDrive/features/tfidf/chunk_4.npz
BERT 特征已保存到 /content/drive/MyDrive/features/bert/chunk_4.npy
TF-IDF 特征已保存到 /content/drive/MyDrive/features/tfidf/chunk_5.npz
BERT 特征已保存到 /content/drive/MyDrive/features/bert/chunk_5.npy
TF-IDF 特征已保存到 /content/drive/MyDrive/features/tfidf/chunk_6.npz
BERT 特征已保存到 /content/drive/MyDrive/features/bert/chunk_6.npy
TF-IDF 特征已保存到 /content/drive/MyDrive/features/tfidf/chunk_7.npz
BERT 特征已保存到 /content/drive/MyDrive/features/bert/chunk_7.npy
TF-IDF 特征已保存到 /content/drive/MyDrive/features/tfidf/chunk_8.npz


In [6]:
import os
import numpy as np
from scipy.sparse import load_npz
import pandas as pd

# 定义目录路径
tfidf_dir = '/content/drive/MyDrive/features/tfidf'
bert_dir = '/content/drive/MyDrive/features/bert'
output_dir = '/content/drive/MyDrive/features/hybrid'

# 检查并创建输出目录
os.makedirs(output_dir, exist_ok=True)

# 获取文件列表
tfidf_files = sorted([f for f in os.listdir(tfidf_dir) if f.endswith('.npz')])
bert_files = sorted([f for f in os.listdir(bert_dir) if f.endswith('.npy')])

print(f"找到 {len(tfidf_files)} 个 TF-IDF 文件")
print(f"找到 {len(bert_files)} 个 BERT 文件")

# 确保文件数量匹配
if len(tfidf_files) != len(bert_files):
    print("错误：TF-IDF和BERT文件数量不匹配！")
else:
    # 遍历所有文件进行特征融合
    for i in range(len(tfidf_files)):
        tfidf_file = os.path.join(tfidf_dir, tfidf_files[i])
        bert_file = os.path.join(bert_dir, bert_files[i])

        try:
            # 加载特征
            print(f"正在处理文件 {i+1}/{len(tfidf_files)}: {tfidf_file} 和 {bert_file}")
            tfidf = load_npz(tfidf_file).toarray()
            bert = np.load(bert_file)

            # 检查样本数量是否匹配
            if tfidf.shape[0] != bert.shape[0]:
                print(f"警告：{tfidf_file} 和 {bert_file} 的样本数量不匹配！")
                continue

            # 特征融合
            hybrid = np.hstack([tfidf, bert])

            # 保存融合特征
            output_file = os.path.join(output_dir, f'hybrid_{i}.npy')
            np.save(output_file, hybrid)
            print(f"✅ 成功保存融合特征到 {output_file}")

        except Exception as e:
            print(f"❌ 处理文件 {tfidf_file} 和 {bert_file} 时出错: {str(e)}")

# 验证融合结果
hybrid_files = sorted([f for f in os.listdir(output_dir) if f.endswith('.npy')])
print(f"融合完成！在 {output_dir} 目录下找到 {len(hybrid_files)} 个混合特征文件")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/features/tfidf'

### Visualization & Abnormal Detection



In [ ]:
import numpy as np
import plotly.express as px
from sklearn.manifold import TSNE
import os

# 定义特征文件路径
output_dir = '/content/drive/MyDrive/features/hybrid'

# 检查 output_dir 是否存在，不存在则创建
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"目录 {output_dir} 不存在，已创建。")

# 初始化一个空列表来存储所有的 hybrid 数据
all_hybrid = []

# 打印 output_dir 中的文件名
filenames = os.listdir(output_dir)
print(f"目录 {output_dir} 中的文件名: {filenames}")

# 遍历所有的 hybrid 文件
for filename in filenames:
    if filename.endswith('.npy'):
        try:
            # 加载每个 hybrid 文件
            hybrid = np.load(os.path.join(output_dir, filename))
            all_hybrid.append(hybrid)
        except Exception as e:
            print(f"加载文件 {filename} 时出错: {e}")

# 打印 all_hybrid 列表的长度
print(f"all_hybrid 列表的长度: {len(all_hybrid)}")

if len(all_hybrid) > 0:
    # 将所有的 hybrid 数据合并为一个数组
    hybrid = np.vstack(all_hybrid)

    # 假设 labels 和 texts 也需要定义
    # 这里需要根据实际情况进行修改
    labels = np.random.randint(0, 2, len(hybrid))
    texts = np.array([f"text_{i}" for i in range(len(hybrid))])

    # 随机采样1%数据
    sample_idx = np.random.choice(len(hybrid), int(0.01 * len(hybrid)), replace=False)
    sample = hybrid[sample_idx]

    # t-SNE降维
    tsne = TSNE(n_components=2, perplexity=30)
    embeddings = tsne.fit_transform(sample)

    # 交互式绘图
    fig = px.scatter(
        x=embeddings[:, 0], y=embeddings[:, 1],
        color=labels[sample_idx],
        hover_name=texts[sample_idx],
        title="威胁检测特征分布"
    )
    fig.write_html('/content/drive/MyDrive/reports/tsne.html')
else:
    print("all_hybrid 列表为空，无法进行拼接操作。")

目录 /content/drive/MyDrive/features/hybrid 不存在，已创建。
目录 /content/drive/MyDrive/features/hybrid 中的文件名: []
all_hybrid 列表的长度: 0
all_hybrid 列表为空，无法进行拼接操作。


### Knowledge Graph

In [ ]:
!pip install openke

from openke.config import Trainer
from openke.module.model import TransE

# 定义实体关系
entities = ['user_001', 'user_002', 'file_A', 'server_X']
relations = ['access', 'modify', 'download']

# 构建三元组
triples = [
    ['user_001', 'access', 'file_A'],
    ['user_002', 'modify', 'server_X']
]

# 训练TransE模型
config = Trainer(save_path = "/content/drive/MyDrive/kg_model")
transe = TransE(
    ent_tot = len(entities),
    rel_tot = len(relations),
    dim = 100
)
trainer = Trainer(model = transe, data_loader = triples)
trainer.run()

### Monitoring Dashboard

In [ ]:
from IPython.display import display, HTML
import time

html = """
<div style="border:2px solid #f0f0f0; padding:10px">
  <h3>任务进度监控</h3>
  <p>TF-IDF特征: <span id="tfidf">0%</span></p >
  <p>BERT编码: <span id="bert">0%</span></p >
  <p>特征融合: <span id="fusion">0%</span></p >
</div>
<script>
function updateProgress(task, percent) {
  document.getElementById(task).innerHTML = percent + "%";
}
</script>
"""

display(HTML(html))

# 在任务代码中添加进度上报
# 在TF-IDF处理循环内
for i, chunk in enumerate(...):
    display(HTML(f'<script>updateProgress("tfidf", {(i+1)/total*100:.1f})</script>'))
    time.sleep(0.1)  # 防止刷新过快